In [214]:
from bs4 import BeautifulSoup
import requests
from config import scrape_key
import pandas as pd
from datetime import date, timedelta 

In [ ]:
stonks = ['AMZN', 'AAPL', 'GOOGL', 'TSLA', 'GME']

In [192]:
query_url = f"https://api.scrapingdog.com/scrape?api_key={scrape_key}&url=https://finance.yahoo.com/quote/AMZN?p=AMZN&.tsrc=fin-srch"
r = requests.get(query_url).text
soup = BeautifulSoup(r,'html.parser')
alldata = soup.find_all("tbody")

try:
 table1 = alldata[0].find_all("tr")
except:
 table1 = None

try:
 table2 = alldata[1].find_all('tr')
except:
 table2 = None

l={}
u=list()

for i in range(0,len(table1)):
 try:
   table1_td = table1[i].find_all('td')
 except:
   table1_td = None
 l[table1_td[0].text] = table1_td[1].text
 u.append(l)
 l={}
    
for i in range(0,len(table2)):
 try:
   table2_td = table2[i].find_all('td')
 except:
   table2_td = None
 l[table2_td[0].text] = table2_td[1].text
 u.append(l)
 l={}

In [193]:
scrape_df = pd.DataFrame(u)
prev_close = scrape_df.iloc[0]['Previous Close']
print(prev_close)

3,203.53


In [178]:
close_date = (date.today() - timedelta(days = 1)).strftime('%m-%d-%Y')
print(close_date)

12-02-2020


In [215]:
stonks = ['AMZN', 'AAPL', 'GOOGL', 'TSLA', 'GME']
stock_perf = []
for stocks in stonks:
    query_url = f"https://api.scrapingdog.com/scrape?api_key={scrape_key}&url=https://finance.yahoo.com/quote/{stocks}?p={stocks}&.tsrc=fin-srch"
    r = requests.get(query_url).text
    soup = BeautifulSoup(r,'html.parser')
    alldata = soup.find_all("tbody")

    try:
     table1 = alldata[0].find_all("tr")
    except:
     table1 = None

    try:
     table2 = alldata[1].find_all('tr')
    except:
     table2 = None

    l={}
    u=list()

    for i in range(0,len(table1)):
     try:
       table1_td = table1[i].find_all('td')
     except:
       table1_td = None
     l[table1_td[0].text] = table1_td[1].text
     u.append(l)
     l={}

    for i in range(0,len(table2)):
     try:
       table2_td = table2[i].find_all('td')
     except:
       table2_td = None
     l[table2_td[0].text] = table2_td[1].text
     u.append(l)
     l={}

#Create Data Frame Rows
    scrape_df = pd.DataFrame(u)
    prev_close = scrape_df.iloc[0]['Previous Close']
    close_date = (date.today() - timedelta(days = 1)).strftime('%m-%d-%Y')
    stock_list = [close_date, stocks, prev_close]
    stock_perf.append(stock_list)
df = pd.DataFram(stock_perf, columns = ['Date'. 'Symbol', 'Close_Value'])
df.

['12-02-2020', 'AMZN', '3,203.53']
['12-02-2020', 'AAPL', '123.08']
['12-02-2020', 'GOOGL', '1,824.97']
['12-02-2020', 'TSLA', '568.82']
['12-02-2020', 'GME', '16.58']


NameError: name 'data_frame_final' is not defined

In [196]:
degrees = [0, 10, 20, 40, 100]
for C in degrees:
    print('list element:', C)
print('The degrees list has', len(degrees), 'elements')

list element: 0
list element: 10
list element: 20
list element: 40
list element: 100
The degrees list has 5 elements


In [213]:
list2 = []
list1 = ['dog', 'cat']
list3 = ['pat','jim']
list2.append(list1)
list2.append(list3)
df = pd.DataFrame(list2, columns = ['Pet1', 'Pet2'])
df.head()

,Pet1,Pet2
0,dog,cat
1,pat,jim
